# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from typing import *

import reader
from bs4 import BeautifulSoup
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, features="lxml")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
("test" "test2")

'testtest2'

In [ ]:
# | export

import rich


class PyNewsReader:
    def __init__(self, dbpath=None, feeds=List[Feed]):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self._reader = reader.make_reader(
            "db.sqlite", plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        self._reader.enable_search()
        self._feed_names = {}

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = self._get_feed_title(e.original_feed_url)
                panel_body = f"Link: [link={e.link}]{e.link}[/link]"+"\n"
                panel_body += str(published_date) + "\n"
                panel_body += "Source: "+feed_title + "\n\n"
                panel_body += strip_html(e.summary) + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=f"[link={e.link}]{e.title}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed

        Args:
            url (str): URL of pynewsreader feed

        Returns:
            str: Display title
        """
        if url in self._feed_names:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ):
        """Get entries in reader.Entry format

        Args:
            read (reader.Entry.read, optional): Filter on `read` status (None, True, False). Defaults to None.
            limit (int, optional): Number of entries to return. Defaults to 10.

        Returns:
            List[reader.Entry]: List of entries
        """
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def add_feed(self, feed: Feed):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        self._reader.add_feed(feed.url, exist_ok=True)
        if feed.name:
            self._feed_names[feed.url] = feed.name

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def show(
        self,
        limit: int = 5,
        read: bool = None,
        important: bool = None,
        mark_as_read: bool = False,
    ):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(
            self._get_entries(read=read, important=important, limit=limit * 2),
            limit=limit,
            mark_as_read=mark_as_read,
        )

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries(
            [self._search_to_entry(i) for i in self._reader.search_entries(query)],
            mark_as_read=mark_as_read,
            limit=limit,
        )

    def mark_important(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_important(entry)

    def mark_unimportant(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_unimportant(entry)

    def tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Tag an entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
            tag_value (Dict, optional): Value of tag. Defaults to None.
        """
        reader.Reader.set_tag(entry, tag_key, tag_value)

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))

## List Feeds

In [ ]:
r.feeds()

['Richochet Media',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Update Feeds

In [ ]:
# r.update()

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(read=False, limit=5, mark_as_read=True)

╭─────────────── ]8;id=364742;https://www.cbc.ca/news/science/biodiversity-montreal-cop27-1.6654447?cmp=rss\Biodiversity needs same protection as climate, say scientists, activists at COP27]8;;\ ───────────────╮
│ Link: ]8;id=621267;https://www.cbc.ca/news/science/biodiversity-montreal-cop27-1.6654447?cmp=rss\https://www.cbc.ca/news/science/biodiversity-montreal-cop27-1.6654447?cmp=rss]8;;\                             │
│ Date: 2022-11-17                                                                                                │
│ Source: CBC | Technology News                                                                                   │
│                                                                                                                 │
│  Civil society groups, Indigenous activists and scientists are standing together at the COP27 climate           │
│ conference in Sharm el-Sheikh, Egypt, and demanding firm action be taken next month at the UN Biodiversity      │
│ Conference that will be hosted in Montreal.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────── ]8;id=76686;https://www.thestar.com/opinion/editorials/2022/11/17/judges-should-be-free-to-impose-conditional-sentences-when-reasonable.html\Judges should be free to impose conditional sentences when reasonable]8;;\ ─────────────────────╮
│ Link:                                                                                                           │
│ ]8;id=185763;https://www.thestar.com/opinion/editorials/2022/11/17/judges-should-be-free-to-impose-conditional-sentences-when-reasonable.html\https://www.thestar.com/opinion/editorials/2022/11/17/judges-should-be-free-to-impose-conditional-sentences-whe]8;;\ │
│ ]8;id=185763;https://www.thestar.com/opinion/editorials/2022/11/17/judges-should-be-free-to-impose-conditional-sentences-when-reasonable.html\n-reasonable.html]8;;\                                                                                               │
│ Date: 2022-11-17                                                                                                │
│ Source: Toronto Star | Editorials                                                                               │
│                                                                                                                 │
│ It costs about $150,000 per year to house one inmate in a federal prison. That’s just one reason to pursue      │
│ alternatives to incarceration for offenders who don’t represent a danger to the public.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ ]8;id=66126;https://www.thestar.com/news/investigations/2022/11/17/they-recently-bought-greenbelt-land-that-was-undevelopable-now-the-ford-government-is-poised-to-remove-protections-and-these-developers-stand-to-profit.html\They recently bought Greenbelt land that was undevelopable. Now the Ford government is poised to remove protec]8;;\─╮
│ Link:                                                                                                           │
│ ]8;id=35881;https://www.thestar.com/news/investigations/2022/11/17/they-recently-bought-greenbelt-land-that-was-undevelopable-now-the-ford-government-is-poised-to-remove-protections-and-these-developers-stand-to-profit.html\https://www.thestar.com/news/investigations/2022/11/17/they-recently-bought-greenbelt-land-that-was-undevelopab]8;;\ │
│ ]8;id=35881;https://www.thestar.com/news/investigations/2022/11/17/they-recently-bought-greenbelt-land-that-was-undevelopable-now-the-ford-government-is-poised-to-remove-protections-and-these-developers-stand-to-profit.html\le-now-the-ford-government-is-poised-to-remove-protections-and-these-developers-stand-to-profit.html]8;;\            │
│ Date: 2022-11-17                                                                                                │
│ Source: Toronto Star | Investigations                                                                           │
│                                                                                                                 │
│ An association representing developers said the GTA is in “the midst of a housing crisis” and more homes are    │
│ needed to meet increased demand.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── ]8;id=740175;https://www.macleans.ca/technology-3/the-big-idea-solving-canadas-supply-chain-woes/\The Big Idea: Airships are back, and they’re better than ever]8;;\ ─────────────────────────╮
│ Link: ]8;id=874757;https://www.macleans.ca/technology-3/the-big-idea-solving-canadas-supply-chain-woes/\https://www.macleans.ca/technology-3/the-big-idea-solving-canadas-supply-chain-woes/]8;;\                      │
│ Date: 2022-11-16                                                                                                │
│ Source: Macleans                                                                                                │
│                                                                                                                 │
│ With a bit of investment, they can help solve our supply chain woes                                             │
│ The post The Big Idea: Airships are back, and they’re better than ever appeared first on Macleans.ca.           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── ]8;id=498469;https://www.cbc.ca/news/science/artemis-1-third-attempt-1.6651139?cmp=rss\NASA finally launches Artemis mission to the moon]8;;\ ───────────────────────────────╮
│ Link: ]8;id=189526;https://www.cbc.ca/news/science/artemis-1-third-attempt-1.6651139?cmp=rss\https://www.cbc.ca/news/science/artemis-1-third-attempt-1.6651139?cmp=rss]8;;\                                 │
│ Date: 2022-11-15                                                                                                │
│ Source: CBC | Technology News                                                                                   │
│                                                                                                                 │
│  It was a nail-biter, but NASA finally got the first rocket in its Artemis mission off the ground. The rocket   │
│ lifted off at 1:47 a.m. ET, lighting up the early morning sky in Cape Canaveral, Fla.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Search Entries

In [ ]:
r.search("fusion")

╭─────────────────── ]8;id=453890;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\Why burning plasma could be the next milestone in nuclear fusion research]8;;\ ───────────────────╮
│ Link:                                                                                                           │
│ ]8;id=733953;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.668]8;;\ │
│ ]8;id=733953;https://www.cbc.ca/radio/quirks/why-burning-plasma-could-be-the-next-milestone-in-nuclear-fusion-research-1.6687322?cmp=rss\7322?cmp=rss]8;;\                                                                                                    │
│ Date: 2022-12-16                                                                                                │
│ Source: CBC | Technology News                                                                                   │
│                                                                                                                 │
│  Bob McDonald's blog: A laser fusion experiment in the U.S. produced more energy out than went into driving it, │
│ which is an important step. But practical fusion power will likely come from a different technology and is      │
│ still many years away.                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── ]8;id=545516;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\Fusion energy 'breakthrough' revealed by U.S. scientists]8;;\ ────────────────────────────╮
│ Link: ]8;id=468588;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss]8;;\                                  │
│ Date: 2022-12-13                                                                                                │
│ Source: CBC | Technology News                                                                                   │
│                                                                                                                 │
│  U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the          │
│ decades-long quest to harness fusion, the energy that powers the sun and stars.                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── ]8;id=573869;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\What is nuclear fusion and why is it such a big deal?]8;;\ ─────────────────────────────╮
│ Link: ]8;id=446973;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss]8;;\                                │
│ Date: 2022-12-13                                                                                                │
│ Source: CBC | Technology News                                                                                   │
│                                                                                                                 │
│  The U.S. Department of Energy on Tuesday announced a breakthrough in nuclear fusion, a way of producing clean  │
│ energy that scientists have been working on since the 1940s. So what exactly is it, why is it so important and  │
│ what does it mean for the future?                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────── ]8;id=248451;https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss\The cannabis industry has a plastic waste problem — but some are finding solutions]8;;\ ───────────────╮
│ Link: ]8;id=527940;https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss\https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss]8;;\                    │
│ Date: 2022-12-15                                                                                                │
│ Source: CBC | Canada News                                                                                       │
│                                                                                                                 │
│  In this week's issue of our environment newsletter, we look at the plastic waste produced in the cannabis      │
│ industry and what this week's announcement about nuclear fusion means for the future of clean energy.           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# | hide

# To Do:
# * Command line interface
# * User interface
# * Tags

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()